# Collaborative Filtering Recommender Systems

In this project, we will implement collaborative filtering to build a recommender system for movies.

# Outline

- [1 - Notation](#1)
- [2 - Recommender Systems](#2)
- [3 - Movie ratings dataset](#3)
- [4 - Collaborative filtering learning algortim](#4)
    - [4.1 - Cost Function](#4.1)
- [5 - Learning movie recommendations](#5)
- [6 - Recommensations](#6)

# Packages


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

<a name="1"></a>
# 1 - Notation

<figure>
    <img src="./images/Notation.PNG", style="width=150px, height=500px">
</figure>

<a name="2"></a>
# 2 - Recommender Systems

You will implement the collaborative filtering learning algorithm and apply it to a dataset of movie ratings. The goal of a collaborative filtering recommender system is to generate two vectors: For each user, a 'parameter vector' that embodies the movie tastes of a user. For each movie, a feature vector of the same size which embodies some description of the movie. The dot product of the two vectors plus the bias term should produce an estimate of the rating the user might give to that movie.

The diagram below details how these vectors are learned.

<figure>
    <img src="./images/ColabFilterLearn.PNG" style="width=500, height=150">
</figure>

Existing ratings are provided in matrix form as shown. $Y$ contains ratings; 0.5 to 5 inclusive in 0.5 steps. 0 if the movie has not been rated. $R$ has a 1 where movies have been rated. Movies are in rows, users in columns. Each user has a parameter vector $w^{user}$ and bias. Each movie has a feature vector $x^{movie}$. These vectors are simultaneously learned by using the existing user/movie ratings as training data. One training example is shown above: $\mathbf{w}^{(1)} \cdot \mathbf{x}^{(1)} + b^{(1)} = 4$. It is worth noting that the feature vector $x^{movie}$ must satisfy all the users while the user vector $w^{user}$ must satisfy all the movies. This is the source of the name of this approach - all the users collaborate to generate the rating set. 

<figure>
    <img src="./images/ColabFilterLearn.PNG" style="width=500, height=150" >
</figure>

Once the feature vectors and parameters are learned, they can be used to predict how a user might rate an unrated movie. This is shown in the diagram above. The equation is an example of predicting a rating for user one on movie zero.

In this project, you will implement the function `cofiCostFunc` that computes the collaborative filtering objective function. After implementing the objective function, you will use a TensorFlow custom training loop to learn the parameters for collaborative filtering. The first step is to detail the data set and data structures that will be used in this project.

<a name="3"></a>
# 3 - Movie ratings dataset